In [1]:
import numpy as np

In [2]:
class K_Filter:
    def __init__(self, y0, dt = 1/25):
        '''
            Ak = constant
            xh = [xh_k+1|k, xh_k|k]
            P = [P_k+1|k, P_k|k]
        '''
        self.dt = dt
        self.Ak = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0]])
        self.xh = [None, None] 
        self.P = [None, self.inv(np.dot(self.A_k.transpose(),self.A_k))]
        
        
    def inv(self, A):
        return np.linalg.inv(A)
    
    def state_update_extrap(self):
        self.Fk = np.array([[1, 0, np.cos(self.xh[1][3])*self.dt, 0, 0],
                            [0, 1, np.sin(self.xh[1][3])*self.dt, 0, 0],
                            [0, 0, 1, 0, 0],
                            [0, 0, 0, 1, self.dt],
                            [0, 0, 0, 0, 1]])
        self.xh[0] = np.dot(self.Fk, self.xh[1])
        
    def info_mtx_extrap(self):
        self.P[0] = np.dot(self.Fk, np.dot(self.P[1], self.Fk.transpose())) + np.eye(5)
        
    def kalman_gain(self):
        a = np.dot(self.Ak, np.dot(self.P[0], self.Ak.transpose())) + np.eye(2)
        self.KG = np.dot(self.P[0], np.dot(self.Ak.transpose(), self.inv(a)))
        
    def state_update(self, yk1):
        self.xh[1] = self.xh[0] + np.dot(self.KG, (yk1 - np.dot(self.Ak, self.xh[0])))
        
    def info_mtx_update(self):
        self.P[1] = np.dot(np.eye(2) - np.dot(self.KG, self.Ak), self.P[0])